In [1]:
import math
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModels import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

Using TensorFlow backend.


In [2]:
def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model

In [13]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs, generateCrossValidation=True, crossValRatio=0.10)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"])
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

In [5]:
file = open("testFile.txt", "w") 

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

#res = nn_optmize_fun(np.array([30, 1, 125]), selected_features = selected_features, scaler = min_max_scaler, 
#                     datasetNumber = '1', verbose=1, epochs=20, saveToFile = file)

file.close()
#print(res)

In [10]:
file = open("intermediateResults.txt", "w")


#Optimize the parameters for the NN using DE

maxWindowSize = {'1':30, '2':20, '3':30, '4':30}
datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

windowSizeBounds = [1,maxWindowSize[datasetNumber]]
windowStrideBounds = [1,10]
constantRULBounds = [80,140]

bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]




startTime = time.clock()
results = differential_evolution(nn_optmize_fun, bounds, 
                                 args=(selected_features, datasetNumber, min_max_scaler, 0, 20, file, 1),
                                strategy='best1bin', maxiter=10, popsize=10, disp=True)
endTime = time.clock()

file.close()
totalTime = endTime - startTime

Instructions for updating:
keep_dims is deprecated, use keepdims instead
differential_evolution step 1: f(x)= 11.5696
differential_evolution step 2: f(x)= 11.5696
differential_evolution step 3: f(x)= 11.5696
differential_evolution step 4: f(x)= 11.5696
differential_evolution step 5: f(x)= 11.2186
differential_evolution step 6: f(x)= 11.2186
differential_evolution step 7: f(x)= 11.2186
differential_evolution step 8: f(x)= 11.2186
differential_evolution step 9: f(x)= 11.2186
differential_evolution step 10: f(x)= 11.2186


In [11]:
print("Total time {}".format(totalTime))
print(results)

Total time 1323.4547188695046
     fun: 11.218635586550983
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 414
     nit: 10
 success: False
       x: array([ 23.80222429,   2.42969551, 105.57937921])


In [16]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

res = nn_optmize_fun(np.array([25, 2, 105]), selected_features = selected_features, scaler = min_max_scaler, 
                     datasetNumber = '1', verbose=1, epochs=20, saveToFile = None)

print(res)

Creating model
Training model
Epoch 1/20
9141/9141 [==============================] - 1s 64us/step - loss: 6841.9786 - mean_squared_error: 6841.9786
Epoch 2/20
9141/9141 [==============================] - 1s 55us/step - loss: 1820.7925 - mean_squared_error: 1820.7925
Epoch 3/20
9141/9141 [==============================] - 1s 56us/step - loss: 776.8217 - mean_squared_error: 776.8217
Epoch 4/20
9141/9141 [==============================] - 1s 55us/step - loss: 615.7692 - mean_squared_error: 615.7692
Epoch 5/20
9141/9141 [==============================] - 1s 55us/step - loss: 549.4608 - mean_squared_error: 549.4608
Epoch 6/20
9141/9141 [==============================] - 1s 55us/step - loss: 524.5251 - mean_squared_error: 524.5251
Epoch 7/20
9141/9141 [==============================] - 1s 55us/step - loss: 489.4189 - mean_squared_error: 489.4189
Epoch 8/20
9141/9141 [==============================] - 1s 55us/step - loss: 479.3290 - mean_squared_error: 479.3290
Epoch 9/20
9141/9141 [========

In [17]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plot(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()